In [1]:
from konlpy.tag import Twitter

import pandas as pd

import pickle
from math import log2,log10,sqrt
from tqdm import tqdm

In [71]:
with open('artMorphs_final.dat','rb') as file: # 미리 doc별로 저장한 morphs 리스트를 load한다
    morphs = pickle.load(file)

In [2]:
with open('artNouns_final.dat','rb') as file: # 전처리 과정에서 저장한 doc별 nouns의 리스트를 load한다
    nouns = pickle.load(file)

In [74]:
with open('artAd_nouns_final.dat','rb') as file: # 전처리 과정에서 저장한 doc별 명사 동사 형용사 부사 감탄사의 리스트를 load한다
    ad_nouns = pickle.load(file)

TF-IDF 기반으로 중요도 높은 단어를 뽑아보자

In [115]:
tokens = ad_nouns #어떤 품사의 토큰을 사용할지 지정한다 ad_nouns는 [명사, 동사,형용사,부사,감탄사]

In [50]:
tokens = morphs # 모든 품사를 사용하는 옵션

In [116]:
unq_tokens = []
for lst in tokens:
    unq_tokens += lst
unq_tokens = set(unq_tokens)

In [117]:
len(unq_tokens)

82047

In [118]:
doc_tokens = {}
maxCount = {}

for i,lst in enumerate(tokens):
    maxCount[i] = 0
    tmp_tokens = {}
    
    for w in lst:
        if w in tmp_tokens.keys():
            tmp_tokens[w] += 1
        else:
            tmp_tokens[w] = 1

        if maxCount[i] < tmp_tokens[w]:
            maxCount[i] = tmp_tokens[w]
                    
    doc_tokens[i] = tmp_tokens


In [119]:
doc_tokens[0]

{'각': 1,
 '감상할': 1,
 '개관': 1,
 '개최': 2,
 '갤러리': 2,
 '것': 3,
 '계기': 1,
 '고향': 2,
 '곳': 1,
 '공동': 1,
 '과정': 1,
 '관': 2,
 '관람객': 1,
 '관장': 1,
 '광주': 9,
 '구입': 1,
 '기회': 2,
 '기획': 2,
 '기획전': 1,
 '까지': 1,
 '남도': 6,
 '년': 4,
 '다를': 1,
 '다큐': 1,
 '달래는': 1,
 '대표': 1,
 '대표하는': 1,
 '대한': 1,
 '동': 1,
 '되었': 1,
 '된': 1,
 '될': 2,
 '를': 1,
 '많은': 1,
 '명': 1,
 '모아': 1,
 '무대': 1,
 '문화': 4,
 '문화재단': 3,
 '물론': 2,
 '물씬': 1,
 '미술': 1,
 '미술관': 7,
 '바': 1,
 '반추': 1,
 '받은': 1,
 '발전': 1,
 '밝혔': 1,
 '방송': 1,
 '봄': 1,
 '뿌리': 1,
 '사간': 1,
 '상록': 1,
 '서울': 1,
 '선정하여': 1,
 '성격': 1,
 '성은': 1,
 '성장': 1,
 '소중한': 1,
 '수': 1,
 '시립': 6,
 '언론': 1,
 '역사': 1,
 '연구': 2,
 '우리': 1,
 '우현': 1,
 '운영해': 1,
 '월': 3,
 '위해': 3,
 '의미': 1,
 '이미': 1,
 '이번': 4,
 '이사장': 1,
 '인': 1,
 '인의': 1,
 '일': 2,
 '있는': 2,
 '있다': 1,
 '작가': 8,
 '작품': 5,
 '장경': 2,
 '재인식': 1,
 '전': 1,
 '전남': 2,
 '전시': 9,
 '전시관': 1,
 '전시하게': 1,
 '전시하여': 1,
 '점': 1,
 '정서': 1,
 '제작하였': 2,
 '제작한': 1,
 '조명': 1,
 '좋': 1,
 '주제': 1,
 '주최': 1,
 '중앙': 1,
 '지난해': 1,
 '지역': 4,
 '지원

In [120]:
for i,(k,v) in enumerate(doc_tokens.items()):
    if i>10: break
    print("{0} - {1}/{2}".format(k, len(v), maxCount[k])) # 문서id-   등장단어수/가장 높았던 빈도

0 - 119/9
1 - 365/11
2 - 139/12
3 - 239/10
4 - 124/6
5 - 338/19
6 - 201/9
7 - 216/6
8 - 213/14
9 - 252/7
10 - 578/22


In [90]:
#시간이 좀 걸린다
invertedIdx = {}
for token in tqdm(unq_tokens):
    invertedIdx[token] = []
    
    for k,v in doc_tokens.items():
        if token in v:
            invertedIdx[token].append(k)

100%|███████████████████████████████████████████████████████████████████████████| 82047/82047 [08:06<00:00, 168.53it/s]


In [57]:
# with open('invertedIdx_morphs_final.dic','wb') as file:
#     pickle.dump(invertedIdx, file)

In [82]:
# with open('invertedIdx_nouns_final.dic','wb') as file:
#     pickle.dump(invertedIdx, file)

In [91]:
# with open('invertedIdx_adnouns_final.dic','wb') as file:
#     pickle.dump(invertedIdx, file)

In [99]:
with open('invertedIdx_morphs_final.dic','rb') as file: # 전체 형태소로 만든 invertedIdx
    invertedIdx = pickle.load(file)

In [111]:
with open('invertedIdx_nouns_final.dic','rb') as file: # 명사로 만든 invertedIdx
    invertedIdx = pickle.load(file)

In [121]:
with open('invertedIdx_adnouns_final.dic','rb') as file: #ads로 만든 invertedIdx
    invertedIdx = pickle.load(file)

In [122]:
K = 0.5 # K + (1-K)*(f(t,d)/maxf(t,d)) 최솟값 0.5, 최댓값 1 
TF = {}
for k,v in doc_tokens.items():
    tfList = {}
    
    for w in v:
        tfList[w] =  K + (1-K)*(v[w]/maxCount[k])
#         print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
#             w, K, (1-K), v[w], maxCount[k], tfList[w]
#         ))
        
    TF[k] = tfList

In [123]:
docSize = len(tokens)
TFIDF = {}
"""
TF = doc1|{단어1:0.54, 단어2:056 ...} 
"""
for (k,v) in TF.items():
    idfList = {}
    for t in v:
        idf = log2(docSize/len(invertedIdx[t]))
        idfList[t] = v[t] * idf
#         print('{0} | {1} * log({2} / {3}) = {4}'.format(
#             t, v[t], docSize, len(invertedIdx[t]), idfList[t]
#         ))
    
    TFIDF[k] = idfList

In [124]:
with open('TFIDF_adnouns_final.dic','wb') as file: #TFIDF 결과를 저장
    pickle.dump(TFIDF, file)

In [125]:
with open('TFIDF_adnouns_final.dic','rb') as file:
    TFIDF = pickle.load(file)

In [126]:
len(TFIDF)

19099

In [132]:
sortedByWeight = [sorted(dic.items(), key=lambda kv:kv[1],reverse=True) for dic in TFIDF.values()] # 내림차순 정렬해보자

In [133]:
# with open('sortedByWeight_ads.lst','wb') as file:
#     pickle.dump(sortedByWeight,file)

In [134]:
# with open('sortedByWeight_ads.lst','rb') as file:
#     sortedByWeight = pickle.load(file)

In [135]:
sortedByWeight[1][:20] #1번 doc의 TFIDF기반 중요단어 top10을 뽑았다

[('박미경', 7.777133306766285),
 ('이주은', 6.848404094906187),
 ('학익동', 6.66611426294253),
 ('포팅', 6.66611426294253),
 ('희영', 6.490517120276697),
 ('조문', 6.440484456057331),
 ('김희연', 6.216818496861944),
 ('변화됨', 6.120659717487984),
 ('인천광역시', 5.945062574822151),
 ('핏기', 5.945062574822151),
 ('반신', 5.945062574822151),
 ('김유정', 5.893302981455927),
 ('박종호', 5.730428083547992),
 ('생육', 5.439971982882393),
 ('회벽', 5.439971982882393),
 ('허용', 5.325128614255484),
 ('모집', 5.256134717094627),
 ('오신', 5.1395189926389016),
 ('붓고', 5.029750626578894),
 ('기와집', 5.029750626578894)]

In [136]:
sortedByWeight[1][-20:] # 불용어 top10

[('삶', 0.6871417023616964),
 ('등', 0.6818489560507676),
 ('되어', 0.660180959484148),
 ('같은', 0.6379040317544681),
 ('통해', 0.6346705955644649),
 ('다른', 0.582013239706914),
 ('이번', 0.5484680832198059),
 ('대한', 0.5463695366559785),
 ('전시', 0.532013454953647),
 ('우리', 0.4711608741173398),
 ('작품', 0.4411743957399941),
 ('작가', 0.4214223938761793),
 ('하는', 0.26827276369081904),
 ('한', 0.26238492759908116),
 ('된', 0.2438965489182666),
 ('있다', 0.23585668581725905),
 ('있는', 0.19773428860220485),
 ('수', 0.17503404372681505),
 ('그', 0.1420819021212263),
 ('것', 0.13062078682988787)]

다른 텍스트를 가져와 이를 기반으로 검색해보자 즉, tf-idf기반으로 가장 유사도가 높은 전시를 뽑아보자 

In [137]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
"""

query의 token을 구해보자

In [138]:
tw = Twitter()

In [139]:
stopwords = ['그','것','이','은','는']

In [140]:
query_adNouns = {}
maxCount = 0


for w in [tag[0] for tag in tw.pos(query) if tag[1] in ['Noun','Adjective','Adverb','Verb','Exclamation']]:
    if w in stopwords:
          continue
    if w in query_adNouns.keys():
        query_adNouns[w] += 1
    else:
        query_adNouns[w] = 1

    if maxCount < query_adNouns[w]:
        maxCount = query_adNouns[w]

print(query_adNouns)

{'빈센트': 1, '반': 6, '고흐': 6, '대표': 1, '작': 1, '중': 1, '하나로': 1, '꼽히는': 1, '별': 7, '빛나는': 3, '밤': 5, '고갱': 1, '다툰': 1, '뒤': 1, '자신': 1, '귀': 1, '자른': 1, '사건': 1, '이후': 1, '생': 1, '레미': 1, '요양원': 1, '있': 1, '때': 1, '그린': 3, '밤하늘': 2, '무한': 1, '함': 1, '표현하는': 1, '대상': 1, '이었': 1, '이보': 1, '먼저': 1, '제작': 1, '된': 2, '아': 2, '를': 1, '카페': 1, '테라스': 1, '나': 2, '론': 1, '강': 1, '위로': 2, '반짝이는': 1, '정경': 1, '다루었': 1, '동생': 1, '테오': 1, '쓴': 1, '편지': 1, '오늘': 1, '아침': 1, '해': 1, '뜨': 1, '한참': 1, '전': 1, '창문': 1, '통해': 1, '무것': 1, '없고': 1, '아주': 1, '커': 1, '보이는': 1, '샛별': 2, '없는': 1, '시골': 1, '보았': 1, '고': 2, '했': 2, '그림': 2, '가운데': 1, '왼쪽': 1, '있는': 2, '커다란': 1, '흰': 1, '별일': 1, '구름': 2, '대기': 1, '별빛': 1, '달빛': 1, '폭발': 1, '있다': 2, '황량': 1, '짙은': 1, '파란색': 1, '하늘': 4, '세상': 1, '종말': 1, '연상': 2, '케': 1, '하고': 1, '소용돌이': 1, '치': 1, '떠': 1, '달': 1, '둘레': 1, '뿌옇': 1, '무리': 1, '져': 1, '비': 1, '연속': 1, '동': 1, '터치': 1, '그려진': 1, '굽이': 2, '치는': 3, '두꺼운': 1, '붓': 1, '놀림': 1, '불꽃': 1, '같은': 1, '사이프러스': 2, '

In [103]:
# # noun 기준 코드
# query_nouns = {}
# maxCount = 0

# for w in tw.nouns(query):
#     if w in query_nouns.keys():
#         query_nouns[w] += 1
#     else:
#         query_nouns[w] = 1

#     if maxCount < query_nouns[w]:
#         maxCount = query_nouns[w]
    
# print(query_nouns)

쿼리를 한 doc으로 보고 가중치를 구한다, 여기서는 tf만 구했다

In [141]:
query_weight = {}

for k,v in query_adNouns.items():
    query_weight[k] = K + (1-K) * (v/maxCount)
    print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
            k, K, (1-K), v, maxCount, query_weight[k]
        ))

빈센트 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
반 | 0.5 + 0.5 *(6/7)  =  0.9285714285714286
고흐 | 0.5 + 0.5 *(6/7)  =  0.9285714285714286
대표 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
작 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
중 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
하나로 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
꼽히는 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
별 | 0.5 + 0.5 *(7/7)  =  1.0
빛나는 | 0.5 + 0.5 *(3/7)  =  0.7142857142857143
밤 | 0.5 + 0.5 *(5/7)  =  0.8571428571428572
고갱 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
다툰 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
뒤 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
자신 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
귀 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
자른 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
사건 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
이후 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
생 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
레미 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
요양원 | 0.5 + 0.5 *(1/7)  =  0.5714285714285714
있 | 0.5 + 0.5 *(1/7)  =  0.5

In [142]:
#문서별로 norm을 구한다
doc_len = {} #문서1:길이, 문서2:길이 ...

for k,v in TFIDF.items(): #k는 문서id, v는 단어를 키, 가중치를 밸류로 가진 딕셔너리
    sumPow = 0
    
    for t in v:
        sumPow += v[t]**2
    
    doc_len[k] = sqrt(sumPow)

In [143]:
#쿼리의 norm을 구한다
for k,v in query_weight.items(): #k는 문서id, v는 단어를 키, 가중치를 밸류로 가진 딕셔너리
#     print(k,v)
    sumPow += v**2
    
query_len = sqrt(sumPow)
query_len

69.3511142460816

In [144]:
result = {}
for k,v in query_weight.items():
    if k in invertedIdx.keys():
        for docid in invertedIdx[k]:
            if docid in result.keys():
                result[docid] += TFIDF[docid][k] * query_weight[k]
            else:
                result[docid] = TFIDF[docid][k] * query_weight[k]

In [145]:
for k,v in result.items():
    result[k] = v/(doc_len[k]*query_len) # 코사인 유사도

In [146]:
top = sorted(result.items(),reverse = True, key= lambda kv:kv[1])[:10] # 유사도 높은 전시 top 10

In [147]:
for k,v in top:
    print("[{0}] 유사도:{1}".format(k,v)) # 해당문서와 쿼리간의 정렬된 코사인 유사도를 도출

[15274] 유사도:0.013899342468613629
[15561] 유사도:0.01263605010377436
[12724] 유사도:0.012303930419761456
[11311] 유사도:0.01228599632150947
[6504] 유사도:0.01206107692270222
[11317] 유사도:0.012013088265188515
[2129] 유사도:0.01190591309590708
[12141] 유사도:0.01186616896390465
[17928] 유사도:0.011792305866384033
[3306] 유사도:0.011759435889362712


비슷하다고 뽑아놓은 전시...

http://www.arthub.co.kr/sub01/board03_view.htm?No=19337

빈도 기반의 유사도 측정은 한계가 있기에

W2V 결과에 TF-IDF값을 가중치로 곱하는 형태로 사용해보자

-> art2vec.ipynb 참고

In [1]:
# 별개로 아티스트 doc에서도 TFIDF 구하자 

In [207]:
with open('artists_twice_morphs_final.lst','rb') as file:
    artists_twice_tokens = pickle.load(file)

In [187]:
with open('artists_twice_adnouns_final.lst','rb') as file:
    artists_twice_tokens = pickle.load(file)

In [3]:
with open('artists_twice_nouns_final.lst','rb') as file:
    artists_twice_tokens = pickle.load(file)

In [4]:
unq_tokens = []
for tpl in artists_twice_tokens:
    unq_tokens += tpl[1]
unq_tokens = set(unq_tokens)

In [5]:
len(unq_tokens)

51346

In [6]:
artists_twice_tokens[-1][0]

'김태기'

In [7]:
doc_tokens = {}
maxCount = {}

for tpl in artists_twice_tokens:
    artist_name = tpl[0]
    tokens = tpl[1]
    
    maxCount[artist_name] = 0
    tmp_tokens = {}
    
    for w in tokens:
        if w in tmp_tokens.keys():
            tmp_tokens[w] += 1
        else:
            tmp_tokens[w] = 1

        if maxCount[artist_name] < tmp_tokens[w]:
            maxCount[artist_name] = tmp_tokens[w]
                    
    doc_tokens[artist_name] = tmp_tokens

In [8]:
doc_tokens['김태기']

{'가감': 1,
 '가변': 1,
 '가상': 1,
 '가장': 1,
 '각': 1,
 '각자': 1,
 '간다': 1,
 '간의': 1,
 '간접': 1,
 '간판': 1,
 '감': 4,
 '감각': 2,
 '감성': 1,
 '감정': 1,
 '강': 1,
 '강남구': 1,
 '강의': 1,
 '개': 1,
 '개인': 2,
 '개체': 1,
 '갤러리': 4,
 '건축': 1,
 '검색': 1,
 '것': 11,
 '겨': 1,
 '견고': 1,
 '결과': 1,
 '결합': 1,
 '경력': 1,
 '경험': 6,
 '곁': 1,
 '계기': 1,
 '계절': 1,
 '고착': 1,
 '고찰': 1,
 '곳': 1,
 '공간': 6,
 '공감각': 1,
 '공모': 1,
 '공생': 1,
 '공유': 1,
 '공존': 1,
 '과': 1,
 '과장': 1,
 '과정': 3,
 '관계': 4,
 '관철': 1,
 '광범위': 1,
 '구분': 1,
 '구성': 3,
 '구심': 1,
 '구조': 3,
 '구축': 1,
 '권용무': 1,
 '귀속': 2,
 '그': 7,
 '그것': 2,
 '그날': 1,
 '그동안': 1,
 '그리기': 1,
 '금고': 1,
 '기계': 4,
 '기고': 1,
 '기능': 2,
 '기본': 1,
 '기억': 7,
 '기원': 1,
 '기회': 3,
 '기획': 2,
 '긴장감': 1,
 '김소영': 1,
 '김지원': 1,
 '김태기': 2,
 '꿈': 1,
 '끝': 1,
 '나': 7,
 '나약': 1,
 '난': 1,
 '낯': 1,
 '내': 7,
 '내재': 1,
 '낼': 1,
 '냉정': 1,
 '너무': 1,
 '너무나': 1,
 '년': 2,
 '눈': 1,
 '다시': 3,
 '단서': 2,
 '단순화': 1,
 '단어': 1,
 '단체': 2,
 '달리': 1,
 '담론': 1,
 '당혹': 1,
 '대리': 1,
 '대상': 2,
 '대한': 7,
 '대해': 2,
 '더': 1,
 '더욱':

In [9]:
#5분정도 걸린다
invertedIdx = {}
for token in tqdm(unq_tokens):
    invertedIdx[token] = []
    
    for k,v in doc_tokens.items():
        if token in v:
            invertedIdx[token].append(k)

100%|███████████████████████████████████████████████████████████████████████████| 51346/51346 [03:31<00:00, 242.48it/s]


In [168]:
# with open('invertedIdx_artists_morphs_final.dic','wb') as file:
#     pickle.dump(invertedIdx, file)

In [212]:
with open('invertedIdx_artists_morphs_final.dic','rb') as file: 
    invertedIdx = pickle.load(file)

In [195]:
# with open('invertedIdx_artists_adnouns_final.dic','wb') as file:
#     pickle.dump(invertedIdx, file)

In [196]:
with open('invertedIdx_artists_adnouns_final.dic','rb') as file: 
    invertedIdx = pickle.load(file)

In [14]:
K = 0.5 # K + (1-K)*(f(t,d)/maxf(t,d)) 최솟값 0.5, 최댓값 1 
TF = {}
for k,v in doc_tokens.items():
    tfList = {}
    
    for w in v:
        tfList[w] =  K + (1-K)*(v[w]/maxCount[k])
#         print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
#             w, K, (1-K), v[w], maxCount[k], tfList[w]
#         ))
        
    TF[k] = tfList

In [15]:
docSize = len(artists_twice_tokens)
TFIDF = {}
"""
TF = doc1|{단어1:0.54, 단어2:056 ...} 
"""
for (k,v) in TF.items():
    idfList = {}
    for t in v:
        idf = log2(docSize/len(invertedIdx[t]))
        idfList[t] = v[t] * idf
#         print('{0} | {1} * log({2} / {3}) = {4}'.format(
#             t, v[t], docSize, len(invertedIdx[t]), idfList[t]
#         ))
    
    TFIDF[k] = idfList

In [215]:
# with open('TFIDF_artists_morphs_final.dic','wb') as file: #TFIDF 결과를 저장
#     pickle.dump(TFIDF, file)

In [216]:
with open('TFIDF_artists_morphs_final.dic','rb') as file:
    TFIDF = pickle.load(file)

In [204]:
# with open('TFIDF_artists_adnouns_final.dic','wb') as file: #TFIDF 결과를 저장
#     pickle.dump(TFIDF, file) 

In [13]:
with open('TFIDF_artists_adnouns_final.dic','rb') as file:
    TFIDF = pickle.load(file)

In [16]:
# with open('TFIDF_artists_nouns_final.dic','wb') as file: #TFIDF 결과를 저장
#     pickle.dump(TFIDF, file) 

In [17]:
with open('TFIDF_artists_nouns_final.dic','rb') as file:
    TFIDF = pickle.load(file)

In [18]:
TFIDF['김태기'] # 작가별로 TFIDF 가중치를 뽑았다

{'가감': 2.7543070147461113,
 '가변': 2.2783033950980305,
 '가상': 1.1854008634836957,
 '가장': 0.4270496505226697,
 '각': 0.9381160631923519,
 '각자': 0.7543653344637362,
 '간다': 1.1825442738839647,
 '간의': 1.1792928197736094,
 '간접': 1.8837378745191096,
 '간판': 2.5662364634038046,
 '감': 0.7680467568814686,
 '감각': 0.6620544402443116,
 '감성': 0.6246235538714074,
 '감정': 0.6132580361050168,
 '강': 1.1760553504553268,
 '강남구': 3.8010794185368253,
 '강의': 2.4590351842907716,
 '개': 0.6460832104500696,
 '개인': 0.4233689852428781,
 '개체': 1.236136096167019,
 '갤러리': 0.5037582591706631,
 '건축': 1.4056309274133338,
 '검색': 3.1689903201748884,
 '것': 0.024871673228847622,
 '겨': 2.0613256107105666,
 '견고': 1.5723618153718022,
 '결과': 0.5602946680009756,
 '결합': 1.081677485787156,
 '경력': 2.4371525252960025,
 '경험': 0.5036558337777529,
 '곁': 2.039394191685934,
 '계기': 0.8135964792418893,
 '계절': 1.6152649147818445,
 '고착': 2.2491846839840277,
 '고찰': 1.5189276067040647,
 '곳': 0.4570493648997634,
 '공간': 0.22555707105260134,
 '공감각':